# Assignment 3
- Scroll down until you see the "Top 100 user IDs" section

In [1]:
# setup

import os, sys, findspark
spark_path = os.environ['SPARK_HOME']
sys.path.append(spark_path + "/bin")
sys.path.append(spark_path + "/python")
sys.path.append(spark_path + "/python/pyspark/")
sys.path.append(spark_path + "/python/lib")
sys.path.append(spark_path + "/python/lib/pyspark.zip")
sys.path.append(spark_path + "/python/lib/py4j-0.10.9-src.zip")
findspark.init()
import pyspark
number_cores = 4
memory_gb = 8
conf = (pyspark.SparkConf().setMaster('local[{}]'.format(number_cores)).set('spark.driver.memory', '{}g'.format(memory_gb)))
sc = pyspark.SparkContext(conf=conf)

In [2]:
import json, re
from pyspark.sql.functions import *

In [3]:
spark = pyspark.sql.SparkSession(sc)

In [15]:
yelp_biz = sc.textFile('./yelp_academic_dataset_business.json.gz').map( lambda entry: json.loads(entry) )
yelp_reviews = sc.textFile('./yelp_academic_dataset_review.json.gz').map( lambda entry: json.loads(entry) )
yelp_users = sc.textFile('./yelp_academic_dataset_user.json.gz').map( lambda entry: json.loads(entry) )


In [6]:
#yelp_biz.take(1)

''' business variables
    - business_id
    - name
    - city
    - categories
'''

list(yelp_biz.first().keys())

['business_id',
 'name',
 'address',
 'city',
 'state',
 'postal_code',
 'latitude',
 'longitude',
 'stars',
 'review_count',
 'is_open',
 'attributes',
 'categories',
 'hours']

In [7]:
#yelp_reviews.take(1)

''' Reviews variables
    - review_id ?
    - user_id
    - business_id
'''

list(yelp_reviews.first().keys())

['review_id',
 'user_id',
 'business_id',
 'stars',
 'useful',
 'funny',
 'cool',
 'text',
 'date']

In [8]:
#yelp_users.take(1)

''' users variables
    - user_id
    - review_count
    - fans
'''

list(yelp_users.first().keys())

['user_id',
 'name',
 'review_count',
 'yelping_since',
 'useful',
 'funny',
 'cool',
 'elite',
 'friends',
 'fans',
 'average_stars',
 'compliment_hot',
 'compliment_more',
 'compliment_profile',
 'compliment_cute',
 'compliment_list',
 'compliment_note',
 'compliment_plain',
 'compliment_cool',
 'compliment_funny',
 'compliment_writer',
 'compliment_photos']

Identify 100 users with highest number of ratings/fans.  

In [9]:
user_scores = yelp_users.map( lambda entry: ( entry['user_id'], entry['fans'], entry['review_count']) )
df = spark.createDataFrame( user_scores , ['id','fans','review_count'] )
df = df.sort(['review_count','fans'],ascending=[0,1])
df.show()

+--------------------+----+------------+
|                  id|fans|review_count|
+--------------------+----+------------+
|8k3aO-mPeyhbR5HUu...|1271|       14455|
|RtGqdDBvvBCjcu5dU...| 285|       12772|
|hWDybu_KvYLSdEFzG...| 659|       12487|
|Hi10sGSZNxQH3NLyW...|2718|       11112|
|P5bUL3Engv-2z6kKo...| 774|        9875|
|8RcEwGrFIgkt9WQ35...| 287|        7745|
|nmdkHL2JKFx55T3nq...| 842|        7626|
|Xwnf20FKuikiHcSpc...| 275|        6762|
|CxDOIDnH8gp9KXzpB...| 789|        6633|
|HFECrzYDpgbS5EmTB...| 421|        5500|
|kS1MQHYwIfD0462PE...| 431|        5156|
|WG3w_73scm_JUWJ_3...|1733|        5013|
|m07sy7eLtOjVdZ8oN...|2034|        4913|
|Xj0O2l0bp633ebmG4...| 214|        4851|
|bQCHF5rn5lMI9c5kE...| 227|        4845|
|XYSDrIef7g4Gmp3lN...| 417|        4828|
|IucvvxdQXXhjQ4z6O...| 134|        4784|
|wZPizeBxMAyOSl0M0...| 467|        4697|
|U4INQZOPSUaj8hMjL...|1090|        4631|
|bLbSNkLggFnqwNNzz...| 785|        4627|
+--------------------+----+------------+
only showing top

In [10]:
df.show()

+--------------------+----+------------+
|                  id|fans|review_count|
+--------------------+----+------------+
|8k3aO-mPeyhbR5HUu...|1271|       14455|
|RtGqdDBvvBCjcu5dU...| 285|       12772|
|hWDybu_KvYLSdEFzG...| 659|       12487|
|Hi10sGSZNxQH3NLyW...|2718|       11112|
|P5bUL3Engv-2z6kKo...| 774|        9875|
|8RcEwGrFIgkt9WQ35...| 287|        7745|
|nmdkHL2JKFx55T3nq...| 842|        7626|
|Xwnf20FKuikiHcSpc...| 275|        6762|
|CxDOIDnH8gp9KXzpB...| 789|        6633|
|HFECrzYDpgbS5EmTB...| 421|        5500|
|kS1MQHYwIfD0462PE...| 431|        5156|
|WG3w_73scm_JUWJ_3...|1733|        5013|
|m07sy7eLtOjVdZ8oN...|2034|        4913|
|Xj0O2l0bp633ebmG4...| 214|        4851|
|bQCHF5rn5lMI9c5kE...| 227|        4845|
|XYSDrIef7g4Gmp3lN...| 417|        4828|
|IucvvxdQXXhjQ4z6O...| 134|        4784|
|wZPizeBxMAyOSl0M0...| 467|        4697|
|U4INQZOPSUaj8hMjL...|1090|        4631|
|bLbSNkLggFnqwNNzz...| 785|        4627|
+--------------------+----+------------+
only showing top

# Top 100 Reviewers IDs

In [11]:
top100 = df.take(100)
top100_ids = list(map( lambda entry: entry['id'] , top100 ))
top100_ids

['8k3aO-mPeyhbR5HUucA5aA',
 'RtGqdDBvvBCjcu5dUqwfzA',
 'hWDybu_KvYLSdEFzGrniTw',
 'Hi10sGSZNxQH3NLyWSZ1oA',
 'P5bUL3Engv-2z6kKohB6qQ',
 '8RcEwGrFIgkt9WQ35E6SnQ',
 'nmdkHL2JKFx55T3nq5VziA',
 'Xwnf20FKuikiHcSpcEbpKQ',
 'CxDOIDnH8gp9KXzpBHJYXw',
 'HFECrzYDpgbS5EmTBtj2zQ',
 'kS1MQHYwIfD0462PE61IBw',
 'WG3w_73scm_JUWJ_3Lgn0Q',
 'm07sy7eLtOjVdZ8oN9JKag',
 'Xj0O2l0bp633ebmG468aZw',
 'bQCHF5rn5lMI9c5kEwCaNA',
 'XYSDrIef7g4Gmp3lNFVO6A',
 'IucvvxdQXXhjQ4z6Or6Nrw',
 'wZPizeBxMAyOSl0M0zuCjg',
 'U4INQZOPSUaj8hMjLlZ3KA',
 'bLbSNkLggFnqwNNzzq-Ijw',
 'FZTkBavnUWVQRK3ULOxePQ',
 'Eypq5gLLjCapBVVnMw_MyA',
 'syvwUKQJ4OYfmL_ixVLMeQ',
 'cMEtAiW60I5wE_vLfTxoJQ',
 'CQUDh80m48xnzUkx-X5NAw',
 '_BcWyKQL16ndpBdggh2kNA',
 'nzsv-p1O8gCfP3XijfQrIw',
 'GHoG4X4FY8D8L563zzPX5w',
 'E_GAXhVA1_lVC2aFpMQElA',
 'GwlrUwkULm4ZLN-Kum5nag',
 'IwHLuykqqmGVNHJ_UCwXWg',
 'WWnhqRnWWjutMqh-2SzEuQ',
 'jpIGlAym6z88W2xzHiK5_A',
 '7JpPvfDAE_14vA1xgFOFjw',
 'ysCBsXWPB-LAiewVS3jZfQ',
 'xJpvRrk04rb4rgWtMXaPUQ',
 'j14WgRoU_-2ZE1aw1dXrJg',
 

Above are the user ids for the top 100 users

# Top 100 Reviewers Habits
- Extract the reviews of these users and combine it with the business information. 
    - Are they eating across multiple metropolitans? 
    - Is there a preference in restaurant/food style of their reviews? 
    - Can you infer the locations of these users?


In [7]:
'''
business variables
    - business_id
    - name
    - city
    - categories
    
users variables
    - user_id
    - review_count
    - fans
    
Reviews variables
    - review_id
    - user_id
    - business_id
'''
yelp_biz = sc.textFile('./yelp_academic_dataset_business.json').map( lambda entry: json.loads(entry) )
yelp_reviews = sc.textFile('./yelp_academic_dataset_review.json').map( lambda entry: json.loads(entry) )
yelp_users = sc.textFile('./yelp_academic_dataset_user.json').map( lambda entry: json.loads(entry) )


In [8]:

yelp_biz = yelp_biz.map( lambda entry: (entry['business_id'],entry['name'],entry['city'],entry['categories'].split(', ') if entry['categories'] else []) )\
                    .toDF(['business_id','name','city','categories'])
yelp_biz = yelp_biz.filter(yelp_biz.city.isNotNull())


In [6]:
yelp_reviews = yelp_reviews.map( lambda entry: (entry['review_id'],entry['user_id'],entry['business_id']) ) \
                            .toDF(['review_id','user_id','business_id'])

yelp_users = yelp_users.filter(lambda entry: entry['user_id'] in top100_ids).map(lambda entry: (entry['user_id'],entry['review_count'], entry['fans']) ).toDF(['user_id','review_count','fans'])

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2.0 failed 1 times, most recent failure: Lost task 0.0 in stage 2.0 (TID 2, yigserpentia.home, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/users/adrian/spark-3.0.1-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 605, in main
    process()
  File "/users/adrian/spark-3.0.1-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 597, in process
    serializer.dump_stream(out_iter, outfile)
  File "/users/adrian/spark-3.0.1-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/serializers.py", line 271, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/users/adrian/spark-3.0.1-bin-hadoop3.2/python/pyspark/rdd.py", line 1440, in takeUpToNumLeft
    yield next(iterator)
  File "/users/adrian/spark-3.0.1-bin-hadoop3.2/p

In [ ]:
yelp_data = yelp_users.join(yelp_reviews, ['user_id'])
yelp_data = yelp_data.join(yelp_biz, ['business_id'])
yelp_data = yelp_data.drop('review_count','fans','review_id').distinct().orderBy('user_id')

In [274]:
yelp_data.show()

+--------------------+--------------------+--------------------+-------------+--------------------+
|         business_id|             user_id|                name|         city|          categories|
+--------------------+--------------------+--------------------+-------------+--------------------+
|s_VpeEfBsG-sGg6Vy...|-G7Zkl1wIWBBmD0KR...|            Pho Lien|     Montréal|[Restaurants, Foo...|
|-9dmhyBvepc08KPEH...|-G7Zkl1wIWBBmD0KR...|            CN Tower|      Toronto|[Public Services ...|
|oWx8F8SDIhO5Cy3_9...|-G7Zkl1wIWBBmD0KR...|         Tim Hortons|      Lachine|[Donuts, Coffee &...|
|ioUzNer0HpiFZ1KZ1...|-G7Zkl1wIWBBmD0KR...|      Vieux-Montréal|     Montréal|[Public Services ...|
|T-sjqbuprqtupCvEb...|-G7Zkl1wIWBBmD0KR...|         Tim Hortons|      Lachine|      [Food, Bagels]|
|BtTCTHwoIfr3fBQja...|-G7Zkl1wIWBBmD0KR...|           Starbucks|      Toronto|[Coffee & Tea, Food]|
|DrdhxvpwlZGhwF5vT...|-G7Zkl1wIWBBmD0KR...|        Maison India|     Montréal|[Vegan, Halal, Re...|


In [ ]:
yelp_data.cache()

In [504]:
from functools import reduce

def merge_restaurant_frequencies( x,y ):
    cats = x[1]
    caty = y[1]
    
    if "Food" not in cats:
        if "Food" not in caty:
            return (x[0],{"No Food Reviews":0})
        return y
    
    cats = { cat: f + (caty[cat] if cat in caty else 0) for cat,f in cats.items() }
    
    return (x[0], cats)


def max_interesting_food( pairs ):
    #print(pairs[1])
    max_val = ("No Food Reviews",0)
    
    for k,v in pairs[1].items():
        if k in ['Professional Sports Teams','Automotive','Mass Media','Home & Garden','Radio Stations','Electronics','Performing Arts','Restaurants','Food','Arts & Entertainment','Shopping','Television Stations','Event Planning & Services']:
            continue
        
        if max_val[1] < v:
            max_val = k,v
            
    return max_val


def merge_frequencies( x,y ):
    cats = x[1]
    caty = y[1]
    
    cats = { cat: f + (caty[cat] if cat in caty else 0) for cat,f in cats.items() }
    
    return (x[0], cats)


def max_pair( pairs ):
    max_val = ("No city",0)
    
    for k,v in pairs[1].items():
        if max_val[1] < v:
            max_val = k,v
            
    return max_val

yelp_cat_freqs = yelp_data.rdd.map(lambda row: (row['user_id'], { cat:1 for cat in row['categories']}) ).mapValues(dict).collect()
yelp_city_freqs = yelp_data.rdd.map(lambda row: (row['user_id'], {row['city']:1}) ).mapValues(dict).collect()


[('-G7Zkl1wIWBBmD0KRy_sCw', {'Saint-Laurent': 1}),
 ('-G7Zkl1wIWBBmD0KRy_sCw', {'Dorval': 1}),
 ('-G7Zkl1wIWBBmD0KRy_sCw', {'Montréal': 1}),
 ('-G7Zkl1wIWBBmD0KRy_sCw', {'Toronto': 1}),
 ('-G7Zkl1wIWBBmD0KRy_sCw', {'Toronto': 1}),
 ('-G7Zkl1wIWBBmD0KRy_sCw', {'Montreal': 1}),
 ('-G7Zkl1wIWBBmD0KRy_sCw', {'Vaughan': 1}),
 ('-G7Zkl1wIWBBmD0KRy_sCw', {'Toronto': 1}),
 ('-G7Zkl1wIWBBmD0KRy_sCw', {'Toronto': 1}),
 ('-G7Zkl1wIWBBmD0KRy_sCw', {'Montréal': 1}),
 ('-G7Zkl1wIWBBmD0KRy_sCw', {'Montréal': 1}),
 ('-G7Zkl1wIWBBmD0KRy_sCw', {'Montréal': 1}),
 ('-G7Zkl1wIWBBmD0KRy_sCw', {'Lachine': 1}),
 ('-G7Zkl1wIWBBmD0KRy_sCw', {'Toronto': 1}),
 ('-G7Zkl1wIWBBmD0KRy_sCw', {'Saint-Laurent': 1}),
 ('-G7Zkl1wIWBBmD0KRy_sCw', {'Toronto': 1}),
 ('-G7Zkl1wIWBBmD0KRy_sCw', {'Thornhill': 1}),
 ('-G7Zkl1wIWBBmD0KRy_sCw', {'Montréal': 1}),
 ('-G7Zkl1wIWBBmD0KRy_sCw', {'Lachine': 1}),
 ('-G7Zkl1wIWBBmD0KRy_sCw', {'Montréal': 1}),
 ('-G7Zkl1wIWBBmD0KRy_sCw', {'Toronto': 1}),
 ('-G7Zkl1wIWBBmD0KRy_sCw', {'Vaugh

In [517]:

for uid in top100_ids:

    cats_filtered = list( filter( lambda item: uid == item[0], yelp_cat_freqs) )
    cats_reduced = list( reduce( merge_restaurant_frequencies, cats_filtered ) )
    most_eaten = max_interesting_food( cats_reduced )
    
    cities_filtered = list( filter( lambda item: uid == item[0], yelp_city_freqs) )
    cities_reduced = list( reduce( merge_frequencies, cities_filtered ) )
    main_city = max_pair( cities_reduced )
    
    print("{:16s} | {:26s} : {:4d} cities  |  Primary : {:16s} [{:4d}]".format(
            str(uid), most_eaten[0], len(cities_filtered), str(main_city[0]), main_city[1]
        )) 
    

8k3aO-mPeyhbR5HUucA5aA | No Food Reviews            :    7 cities  |  Primary : Las Vegas        [   6]
RtGqdDBvvBCjcu5dUqwfzA | Coffee & Tea               :  130 cities  |  Primary : Toronto          [  11]
hWDybu_KvYLSdEFzGrniTw | Convenience Stores         : 1496 cities  |  Primary : Charlotte        [1014]
Hi10sGSZNxQH3NLyWSZ1oA | Specialty Food             :  360 cities  |  Primary : Charlotte        [  52]
P5bUL3Engv-2z6kKohB6qQ | Breweries                  :    5 cities  |  Primary : Chambly          [   1]
8RcEwGrFIgkt9WQ35E6SnQ | Desserts                   :  102 cities  |  Primary : Las Vegas        [  59]
nmdkHL2JKFx55T3nq5VziA | Juice Bars & Smoothies     :  197 cities  |  Primary : Las Vegas        [ 183]
Xwnf20FKuikiHcSpcEbpKQ | Coffee & Tea               :  382 cities  |  Primary : Calgary          [ 222]
CxDOIDnH8gp9KXzpBHJYXw | Coffee & Tea               : 4050 cities  |  Primary : Richmond Hill    [ 372]
HFECrzYDpgbS5EmTBtj2zQ | Coffee & Tea               :  140 citie

3mNz5nQFTIBQm0oU5mBR0w | Nightlife                  :   42 cities  |  Primary : Las Vegas        [  29]
-G7Zkl1wIWBBmD0KRy_sCw | Nightlife                  :   37 cities  |  Primary : Saint-Laurent    [   3]
ZRnsUs9q6DbeJbOLjCY4ZA | Coffee & Tea               :  296 cities  |  Primary : Charlotte        [   5]
1xJwMqmvVXPwY7yEVQIc-A | Colombian                  :   13 cities  |  Primary : Las Vegas        [  10]
I9vkjKs1O5Ctf5gy0ry4yg | Breakfast & Brunch         :   70 cities  |  Primary : Las Vegas        [  25]
om5ZiponkpRqUNa3pVPiRg | Nightlife                  :   73 cities  |  Primary : Scottsdale       [  26]
eSlOI3GhroEtcbaD_nFXJQ | Breakfast & Brunch         :   76 cities  |  Primary : Glendale         [  12]
tgrs5VQ6thhouiEtMU-N_w | Nightlife                  :    8 cities  |  Primary : Las Vegas        [   6]
WWWujmZ95y0a2RR5_kQ9sw | Nightlife                  :   69 cities  |  Primary : Las Vegas        [  41]
0zDHiDbj79uBOy3OAcE25A | Hotels & Travel            :   38 citie

## Results
   As we can see in the formatted output, the top reviewers have a wide variety of tastes and preferences. However, it appears that most of them are centered around Las Vegas, as that is where the majority of the reviewed establishments are based.

# Favorite Resaurant 
- Identify one of your favorite restaurants that is available on Yelp. Search for all reviews and reviewers for this restaurants. 
    - Is this restaurant frequented by non-local reviewers (how do you know)?
    - What are the positive things about this restaurant (study higher-rated reviews)
    - What are the negative things about this restaurant (study lower-rated reviews)

## Choosing a location
- Unfortunately, about every restaurant that I thought of was not included in this dataset. So, I decided instead to choose a Starbucks in PA at random.
- I chose a Starbucks in "Bridgeville, PA" with the id "NeM7anGnTOTn7sEJavS3sw"

In [535]:
!grep "Starbucks" ./yelp_academic_dataset_business.json | grep "\"PA\""

{"business_id":"mg-6bfmZdM2AcdD8fXQkZA","name":"Starbucks","address":"1279 Camp Horne Rd","city":"Pittsburgh","state":"PA","postal_code":"15237","latitude":40.535904,"longitude":-80.068405,"stars":2.5,"review_count":3,"is_open":1,"attributes":{"BikeParking":"False","OutdoorSeating":"False","WiFi":"'free'","BusinessParking":"{'garage': False, 'street': False, 'validated': False, 'lot': False, 'valet': False}","BusinessAcceptsCreditCards":"True","RestaurantsPriceRange2":"1"},"categories":"Coffee & Tea, Food","hours":{"Monday":"8:0-20:0","Tuesday":"8:0-20:0","Wednesday":"8:0-20:0","Thursday":"8:0-20:0","Friday":"8:0-20:0","Saturday":"8:0-20:0","Sunday":"8:0-20:0"}}
{"business_id":"995cDn3wIwi4pw4N_uLSqw","name":"Starbucks","address":"1000 Ross Park Mall Drive, K108B","city":"Pittsburgh","state":"PA","postal_code":"15237","latitude":40.542894,"longitude":-80.008204,"stars":4.0,"review_count":5,"is_open":1,"attributes":{"RestaurantsPriceRange2":"2","BusinessAcceptsCreditCards":"True","Outdo

{"business_id":"z7ma746pVx6W8Muy_mnihQ","name":"Starbucks","address":"360 E Waterfront Dr","city":"West Homestead","state":"PA","postal_code":"15120","latitude":40.410638,"longitude":-79.910774,"stars":5.0,"review_count":3,"is_open":1,"attributes":{"BusinessParking":"{'garage': False, 'street': False, 'validated': False, 'lot': False, 'valet': False}"},"categories":"Coffee & Tea, Food","hours":{"Monday":"8:0-20:0","Tuesday":"8:0-20:0","Wednesday":"8:0-20:0","Thursday":"8:0-20:0","Friday":"8:0-20:0","Saturday":"8:0-20:0","Sunday":"8:0-20:0"}}
{"business_id":"RZsk15TxnePHuL9l9jL51g","name":"Starbucks","address":"301 S. Hills Village","city":"Bethel Park","state":"PA","postal_code":"15241","latitude":40.343526,"longitude":-80.055357,"stars":2.0,"review_count":6,"is_open":1,"attributes":{"RestaurantsTakeOut":"True","BusinessAcceptsCreditCards":"True","OutdoorSeating":"False"},"categories":"Coffee & Tea, Food","hours":{"Monday":"7:30-21:30","Tuesday":"7:30-21:30","Wednesday":"7:30-21:30","T

{"business_id":"-E9UVSXSa0ZQA1I60BTzPg","name":"Starbucks","address":"301 Grant Street, Unit R-105","city":"Pittsburgh","state":"PA","postal_code":"15219","latitude":40.438164,"longitude":-79.998412,"stars":4.0,"review_count":9,"is_open":1,"attributes":{"BikeParking":"True","BusinessAcceptsCreditCards":"True","RestaurantsTakeOut":"True","WiFi":"'free'","RestaurantsPriceRange2":"2","OutdoorSeating":"False","BusinessParking":"{'garage': False, 'street': False, 'validated': False, 'lot': False, 'valet': False}"},"categories":"Coffee & Tea, Food","hours":{"Monday":"5:30-18:0","Tuesday":"5:30-18:0","Wednesday":"5:30-18:0","Thursday":"5:30-18:0","Friday":"5:30-18:0"}}
{"business_id":"CS7DuVN2YkWDU7aoeFj-og","name":"Starbucks","address":"225 Summit Park Dr","city":"Pittsburgh","state":"PA","postal_code":"15275","latitude":40.4481240171,"longitude":-80.1794339745,"stars":3.5,"review_count":14,"is_open":0,"attributes":{"Ambience":"{'romantic': False, 'intimate': False, 'classy': False, 'hipster

{"business_id":"iwPtdeoMCe9KBKHYEIHvSA","name":"Starbucks","address":"114 Bakery Square Blvd","city":"Pittsburgh","state":"PA","postal_code":"15206","latitude":40.4566213257,"longitude":-79.9156211105,"stars":5.0,"review_count":7,"is_open":1,"attributes":{"BusinessParking":"{'garage': True, 'street': False, 'validated': False, 'lot': False, 'valet': False}"},"categories":"Coffee & Tea, Food, Coffee Roasteries","hours":null}
{"business_id":"t3OkTBKYbFw3ool219WirA","name":"Starbucks","address":"112 Washington Pl","city":"Pittsburgh","state":"PA","postal_code":"15219","latitude":40.439642,"longitude":-79.99258,"stars":3.0,"review_count":4,"is_open":1,"attributes":{"BusinessAcceptsCreditCards":"True"},"categories":"Coffee & Tea, Food","hours":{"Monday":"6:0-17:0","Tuesday":"6:0-17:0","Wednesday":"6:0-17:0","Thursday":"6:0-17:0","Friday":"6:0-17:0","Saturday":"7:0-13:0","Sunday":"7:0-13:0"}}
{"business_id":"yHx_F5tpor6_26tkp5T4UA","name":"Starbucks","address":"4080 Washington Road","city":"

{"business_id":"qxcZa0Y-6k_Kwc_2xjXEYQ","name":"Starbucks","address":"100 Hazel Lane","city":"Sewickley","state":"PA","postal_code":"15143","latitude":40.543004,"longitude":-80.192614,"stars":4.0,"review_count":4,"is_open":1,"attributes":{"BusinessAcceptsCreditCards":"True","OutdoorSeating":"False","RestaurantsPriceRange2":"2","BikeParking":"True","RestaurantsTakeOut":"True","WiFi":"'free'","BusinessParking":"{'garage': False, 'street': False, 'validated': False, 'lot': False, 'valet': False}"},"categories":"Coffee & Tea, Food, Cafes, Restaurants","hours":{"Monday":"5:0-22:0","Tuesday":"5:0-22:0","Wednesday":"5:0-22:0","Thursday":"5:0-22:30","Friday":"5:0-21:30","Saturday":"5:0-21:30","Sunday":"5:0-22:0"}}
{"business_id":"guBu6tjm6gAQumRJ1BjF4g","name":"Starbucks","address":"428 Clairton Blvd.","city":"Pleasant Hills","state":"PA","postal_code":"15236","latitude":40.33534058,"longitude":-79.9476496667,"stars":4.0,"review_count":14,"is_open":1,"attributes":{"WiFi":"'free'","RestaurantsT

{"business_id":"NeM7anGnTOTn7sEJavS3sw","name":"Starbucks","address":"1597 Washington Pike, Space A-1","city":"Bridgeville","state":"PA","postal_code":"15017","latitude":40.381477,"longitude":-80.09568,"stars":4.5,"review_count":26,"is_open":1,"attributes":{"BikeParking":"True","Caters":"False","OutdoorSeating":"True","BusinessParking":"{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}","WiFi":"'free'","BusinessAcceptsCreditCards":"True","RestaurantsDelivery":"False","Alcohol":"'none'","RestaurantsPriceRange2":"1","RestaurantsTakeOut":"True"},"categories":"Food, Coffee & Tea","hours":{"Monday":"5:0-21:30","Tuesday":"5:0-21:30","Wednesday":"5:0-21:30","Thursday":"6:0-22:0","Friday":"5:0-22:0","Saturday":"5:0-22:0","Sunday":"6:0-21:0"}}


In [537]:
!grep "

" ./yelp_academic_dataset_review.json

{"review_id":"LYV74NGv9QgN5a90TiLhuA","user_id":"I2tFHmA64c-jcd_fBbwtKQ","business_id":"dd2lOVfpM3nLbcJs3Gq0HQ","stars":5.0,"useful":0,"funny":0,"cool":1,"text":"I wish I remembered the girl's name, but she made my night! I asked for the Pink Drink and she surprised me when she knew how to make it without me pulling up the recipe!\n\nI was randomly telling her about my experience at another location where they were out of strawberries for the Strawberry Acai Refresher on three different days. And how the third attempt, the lady was a bit rude in answering my questions about when they would have the strawberries back in stock.\n\nThe barista handed me my drink and surprised me when she told me the drink was on them. Definitely had me feeling confused why my drink was free (not that it's bad) when she told me the drink was suppose to make it up for my experience at the other location.\n\nReally wish I knew her name, but she definitely made me feel that this is how Starbucks values a cust

{"review_id":"wU1RVIF2VlYXHu-QK23wCg","user_id":"EOxmDIA_t5lcBWeYyOv9vw","business_id":"dd2lOVfpM3nLbcJs3Gq0HQ","stars":1.0,"useful":0,"funny":0,"cool":0,"text":"Rudest employees ever. I could care less if you make a mistake, people are only human! But their strategy of handling THEIR mistakes are blaming you, demeaning you In front of other customers \/ employees, and talking about you like you're not even there. I go everywhere with a smile on my place, I wait patiently, I say thank you. There's no reason why a mistake made by the employees in which I smile and say no problem should be handled by them putting me down. Go to this piece of s**t location if you like a bunch of woman putting other women down because they're miserable with their lives. Otherwise, find another location.","date":"2018-07-30 21:52:39"}
{"review_id":"dw8-MdgKWzTJ021OppXt-g","user_id":"VDxXzmFEjurvR6pfp0DKSg","business_id":"dd2lOVfpM3nLbcJs3Gq0HQ","stars":2.0,"useful":0,"funny":0,"cool":0,"text":"This is my us

In [155]:
specified_business_id = 'NeM7anGnTOTn7sEJavS3sw'

In [163]:
sb_reviews = sc.textFile('./yelp_academic_dataset_review.json').map( lambda entry: json.loads(entry) )

sb_reviews = sb_reviews.filter(lambda entry: entry['business_id'] == specified_business_id) \
                        .map( lambda entry: (entry['review_id'],entry['user_id'],entry['business_id']) ) \
                        .toDF(['review_id','user_id','business_id'])
sb_reviewers = sb_reviews.rdd.map(lambda entry: entry['user_id'])

In [164]:
sb_reviewers = sb_reviewers.collect()
sb_reviewers

['yV4dun0_ALnZ6LQIsrCu_Q',
 'HIlTgXL9e3lOJZDQvQKYeQ',
 'nOcP-0PIg4REFX4JeR49aw',
 'HvDBT2whHCalPX7eWgmrqw',
 'km7qrqi424cIp3dqkPdDvQ',
 'HxyRoZiLH8Qn9yeVm72yJw',
 'uPVqAccCotgJKtJ7mJ8GjQ',
 'PsH5Qi8R69WAlhNv8mXWRA',
 'eJbN3c794MNnV5cKHBoTnQ',
 'BbUvnx6Q4Pyrh9cTl1CHQw',
 'Nx3Y_sXwqrSfwWS8LsNiRg',
 'avF-Hwruq-L2xc1Qklr6yw',
 'cre9TUf19FBuTYxndrMaeg',
 'Fg0RIeqM0-nccUwpc9BozQ',
 'Xh7tksOlF9VTaz3tMJv1XA',
 'QYKexxaOJQlseGWmc6soRg',
 'MoFYvOoufOMyVTmUFAhRxQ',
 'ZasBK-gfOFu3kZg_DTUPDw',
 'iVNGt3ZwlZtmfYhV0xRejw',
 'UsmTxWbobLsI6WR1Db0W7A',
 'EzVLcZ6QwLLh2oD2e_dnLg',
 'rFeUNmWzV2sERSTc7-5Ujw',
 '1BZnZoixXXitgFaXCU3tOA',
 '_SqZa-TOuBWE4xJ0opwbRg',
 'hg71CGHy9bwpgY8cGVVg4w',
 'DAvxZtjoGPsgmB57ZXoN2Q']

In [165]:
yelp_users = sc.textFile('./yelp_academic_dataset_user.json').map( lambda entry: json.loads(entry) )

In [166]:
sb_users = yelp_users.filter(lambda entry: entry['user_id'] in sb_reviewers)
sb_users = sb_users.map(lambda entry: (entry['user_id'],entry['review_count'], entry['fans']) ).toDF(['user_id','review_count','fans'])
sb_data = sb_users.join(yelp_reviews, ['user_id'])
sb_data = sb_data.join(yelp_biz, ['business_id'])
sb_data = sb_data.drop('review_count','fans','review_id').distinct().orderBy('user_id')

In [167]:
from functools import reduce

sb_city_freqs = sb_data.rdd.filter(lambda entry: entry['user_id'] in sb_reviewers)


In [168]:
sb_city_freqs = sb_city_freqs.map(lambda row: (row['user_id'], {row['city']:1}) ).collect()


In [169]:

def merge_frequencies( x,y ):
    
    key = x[0]
    merger = x[1]
        
    for city,freq in y[1].items():
        if city in merger:
            merger[city] += freq
        else:
            merger[city] = freq
       
    return (key,merger)
        

def max_item( entry ):
    k,values = entry[:2]
    
    max_val = ("No city",0)
    
    for k,v in values.items():
        if max_val[1] < v:
            max_val = k,v
            
    return max_val


for uid in sb_reviewers:
    print("\n",uid,end="")
    
    cities_filtered = list( filter( lambda item: uid in item, sb_city_freqs) )
    
    if cities_filtered:
        cities_reduced = reduce( merge_frequencies, cities_filtered )
    
        main_city = max_item( cities_reduced )
        
        print("\t {:4d} reviews  |  Primary : {:16s} [{:4d}]".format( len(cities_filtered), str(main_city[0]), main_city[1]),end="") 
        
    


 yV4dun0_ALnZ6LQIsrCu_Q	  169 reviews  |  Primary : Tempe            [  56]
 HIlTgXL9e3lOJZDQvQKYeQ	   20 reviews  |  Primary : Pittsburgh       [   9]
 nOcP-0PIg4REFX4JeR49aw	   88 reviews  |  Primary : Pittsburgh       [  69]
 HvDBT2whHCalPX7eWgmrqw	  183 reviews  |  Primary : Pittsburgh       [ 142]
 km7qrqi424cIp3dqkPdDvQ	  105 reviews  |  Primary : Pittsburgh       [  70]
 HxyRoZiLH8Qn9yeVm72yJw	    5 reviews  |  Primary : Carnegie         [   2]
 uPVqAccCotgJKtJ7mJ8GjQ	    5 reviews  |  Primary : Pittsburgh       [   3]
 PsH5Qi8R69WAlhNv8mXWRA	    7 reviews  |  Primary : Pittsburgh       [   4]
 eJbN3c794MNnV5cKHBoTnQ	  115 reviews  |  Primary : Pittsburgh       [  67]
 BbUvnx6Q4Pyrh9cTl1CHQw	    6 reviews  |  Primary : Bridgeville      [   3]
 Nx3Y_sXwqrSfwWS8LsNiRg	   74 reviews  |  Primary : Pittsburgh       [  41]
 avF-Hwruq-L2xc1Qklr6yw	    5 reviews  |  Primary : Carnegie         [   3]
 cre9TUf19FBuTYxndrMaeg	   15 reviews  |  Primary : Pittsburgh       [   6]
 Fg0RIeqM0-

# Results
Using the chosen Starbucks, it seems that the reviewers are heavily concentrated in Pittsburgh.

Looking at the reviews, people appreciate the location, and the quality of service, which, as it is a well-reputed international chain coffee-shop, is to be expected. Not many bad reviews, only complaining about common poor parking-lot ettiquete and poor service experiences, which considering the average wage of chain-restaurants, is neither uncommon or surprising. 
It's pretty much what one would expect
